<a href="https://colab.research.google.com/github/kylampiima/FYP/blob/main/WIP_Data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pydub

In [ ]:
from pydub import AudioSegment

In [ ]:
audio1 = AudioSegment.from_file("harvard.wav", format="wav")

In [ ]:
sample_rate = 1411
num_samples = sample_rate * 2
trimmed_audio = audio1[:num_samples]

In [ ]:
trimmed_audio.export("outpu.wav", format="wav")

audio2 = AudioSegment.from_file("output.wav", format="wav")

In [ ]:
concatenated_audio = audio1 + audio2

In [32]:
import struct
import wave

# Open the wave file
input_file_path = "harvard.wav"
output_file_path = "output.wav"
output_file_path2 = "output2.wav"

wave_file = wave.open(input_file_path, "r")

# Get the number of frames and parameters
n_frames = wave_file.getnframes()
n_channels = wave_file.getnchannels()
sample_width = wave_file.getsampwidth()

# Ensure we are working with 16-bit PCM audio
if sample_width != 2:
    print("This code only works with 16-bit PCM audio.")
else:
    first_non_silent_frame_index = None

    # Iterate through frames to find the first non-silent frame
    for i in range(n_frames):
        # Read a single frame (2 bytes per channel for 16-bit audio)
        current_frame = wave_file.readframes(1)

        # Check for silence
        silent = True

        # Check the length of the current frame
        if len(current_frame) < n_channels * sample_width:
            print(f"current_frame is too short: {len(current_frame)} bytes")
            break  # Exit if not enough data is available

        # Unpack the signed values from the current frame
        unpacked_signed_values = struct.unpack("<" + "h" * n_channels, current_frame)

        # Check if any channel indicates sound
        for value in unpacked_signed_values:
            if abs(value) > 500:  # Adjust the threshold as needed
                silent = False
                break

        # If we found a non-silent frame, store its index and break
        if not silent:
            first_non_silent_frame_index = i
            break

    # If a non-silent frame was found
    if first_non_silent_frame_index is not None:
        # Move the file pointer to the first non-silent frame
        wave_file.setpos(first_non_silent_frame_index)

        # Read the remaining frames
        remaining_frames = wave_file.readframes(n_frames - first_non_silent_frame_index)
        first_non_silent = wave_file.readframes(first_non_silent_frame_index)

        # Create a new wave file to save the remaining frames
        with wave.open(output_file_path, "w") as output_wave_file:
            # Set parameters for the new wave file
            output_wave_file.setnchannels(n_channels)
            output_wave_file.setsampwidth(sample_width)
            output_wave_file.setframerate(wave_file.getframerate())
            output_wave_file.writeframes(remaining_frames)

        # Create a new wave file to save the first none silent frame
        with wave.open(output_file_path2, "w") as output_wave_file2:
            # Set parameters for the new wave file
            output_wave_file2.setnchannels(n_channels)
            output_wave_file2.setsampwidth(sample_width)
            output_wave_file2.setframerate(wave_file.getframerate())
            output_wave_file2.writeframes(first_non_silent)

        print(f"Exported frames starting from frame {first_non_silent_frame_index} to '{output_file_path}'")
        print("Exported first none silent frame")
    else:
        print("No non-silent frames found.")

# Close the wave file
wave_file.close()

Exported frames starting from frame 33809 to 'output.wav'
Exported first none silent frame
